## Evaluate.ipynb
#### **This notebook evaluates the accuracy of just one model on the IEMOCAP dataset ground truths.**

- **Transcription Model (S2T):** whisper-large-v3
- Source: [HuggingFace](https://huggingface.co/openai/whisper-large-v3)

## Setup

In [1]:
# Run
SEED = 22
FOLD = 0

# Models
TRANSCRIPTION_MODEL_NAME = "openai/whisper-large-v3"

# Flag to enable parsing of arguments when converted to script. Set true after converting
PYTHON_SCRIPT = False

### For Conversion to .py file

In [2]:
if PYTHON_SCRIPT:

    import argparse

    parser = argparse.ArgumentParser(description="")
    parser.add_argument('--seed',default=2021,type=int)
    parser.add_argument('--fold',type=int,required=True)
    parser.add_argument('--remap',type=bool,required=True)
    parser.add_argument('--threshold',type=float,required=False)
    parser.add_argument('--mode',required=False)
    parser.add_argument('--flip',type=bool,required=False)

    # Parse the arguments
    args = parser.parse_args()

    # Run
    SEED = args.seed
    FOLD = args.fold
    RUN_REMAP = args.remap
    THRESHOLD = args.threshold
    MODE = args.mode
    FLIP = args.flip

### Folders

In [3]:
# Filepaths
OUTPUT_FOLDER = f'./fold_{FOLD}'
LOG_OUTPUT = OUTPUT_FOLDER + f'/fold-{FOLD}.log'

### Dataset Setup

In [4]:
# Dataset
TSV = r'C:\Users\ChenYi\Downloads\AAI3001_Project\labels\IEMOCAP_4.tsv'
AUDIO_DIRECTORY = r'C:\Users\ChenYi\Downloads\AAI3001_Project\small-project\IEMOCAP_full_release_audio'

### Select GPUs (For multi-GPU setup)

In [5]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

### Logger

In [6]:
import logging
def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter("[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s")
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w", encoding="utf-8")
    fh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(fh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    sh = logging.StreamHandler()                                                                                                                                                                                   
    sh.setFormatter(formatter)                                                                                                                                                                                     
    logger.addHandler(sh)                                                                                                                                                                                          
                                                                                                                                                                                                                   
    return logger
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
logger = get_logger(LOG_OUTPUT)

## Imports

In [7]:
import csv
import torch
import random
import librosa
import torchaudio
import contractions
import torch.nn as nn
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datasets import Dataset, DatasetDict
from torchaudio import functional as audioF
from torchaudio.transforms import Resample
from torchaudio.compliance import kaldi
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report, recall_score, accuracy_score
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

[2025-01-20 01:15:33,922][config.py][line:54][INFO] PyTorch version 2.4.1+cu124 available.
[2025-01-20 01:15:33,940][config.py][line:101][INFO] TensorFlow version 2.18.0 available.


### Log Details

In [8]:
logger.info("----- Models -----")
logger.info(f"Speech-To-Text (Transcription) Model: {TRANSCRIPTION_MODEL_NAME}")
logger.info("----- Parameters -----")
logger.info(f"Seed: {SEED}")
logger.info(f"Fold: {FOLD}")
logger.info("--------------------")

[2025-01-20 01:15:39,059][1971636866.py][line:1][INFO] ----- Models -----
[2025-01-20 01:15:39,060][1971636866.py][line:2][INFO] Speech-To-Text (Transcription) Model: openai/whisper-large-v3
[2025-01-20 01:15:39,060][1971636866.py][line:3][INFO] ----- Parameters -----
[2025-01-20 01:15:39,061][1971636866.py][line:4][INFO] Seed: 22
[2025-01-20 01:15:39,062][1971636866.py][line:5][INFO] Fold: 0
[2025-01-20 01:15:39,063][1971636866.py][line:6][INFO] --------------------


### Provided Code

In [9]:
class Pad_trunc_wav(nn.Module):
    def __init__(self, max_len: int = 6*16000):
        super(Pad_trunc_wav, self).__init__()
        self.max_len = max_len
    def forward(self,x):
        shape = x.shape
        length = shape[1]
        if length < self.max_len:
            multiple = self.max_len//length+1
            x_tmp = torch.cat((x,)*multiple, axis=1)
            x_new = x_tmp[:,0:self.max_len]
        else:
            x_new = x[:,0:self.max_len]
        return x_new

In [10]:
def setup_seed(seed=2021):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
setup_seed(SEED)

### Download Required Models

In [11]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    TRANSCRIPTION_MODEL_NAME, torch_dtype=torch_dtype, low_cpu_mem_usage=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(TRANSCRIPTION_MODEL_NAME)

forced_decoder_ids = processor.get_decoder_prompt_ids(language="en", task="transcribe")
model.config.forced_decoder_ids = forced_decoder_ids

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    chunk_length_s=30,
    batch_size=1,
    torch_dtype=torch.float16,
    device="cuda:0"
)

logger.info(F"Speech-To-Text (Transcription) model loaded from {TRANSCRIPTION_MODEL_NAME} successfully")

[2025-01-20 01:15:46,006][361834570.py][line:25][INFO] Speech-To-Text (Transcription) model loaded from openai/whisper-large-v3 successfully


### Dataset & Loading

In [12]:
class Mydataset(Dataset):
    def __init__(self, mode='train', max_len=6, seed=2021, fold=0, data_path=TSV, audio_dir=AUDIO_DIRECTORY):
        self.mode = mode
        data_all = pd.read_csv(data_path, sep='\t')
        SpkNames = np.unique(data_all['speaker'])  # ['Ses01F', 'Ses01M', ..., 'Ses05M']
        self.data_info = self.split_dataset(data_all, fold, SpkNames, mode)
        self.get_audio_dir_path = os.path.join(audio_dir)
        self.pad_trunc = Pad_trunc_wav(max_len * 16000)
         
        # Label encoding
        self.label = self.data_info['label'].astype('category').cat.codes.values
        self.ClassNames = np.unique(self.data_info['label'])
        self.NumClasses = len(self.ClassNames)
        self.weight = 1 / torch.tensor([(self.label == i).sum() for i in range(self.NumClasses)]).float()

    def get_classname(self):
        return self.ClassNames
    
    # Updated split_dataset function using fold
    
    def split_dataset(self, df_all, fold, speakers, mode):
        
        spk_len = len(speakers)
        test_idx = np.array(df_all['speaker']==speakers[fold%spk_len])
        if fold%2==0:
            val_idx = np.array(df_all['speaker']==speakers[(fold+1)%spk_len])
        else:
            val_idx = np.array(df_all['speaker']==speakers[(fold-1)%spk_len])
        train_idx = True^(test_idx+val_idx)
        train_data_info = df_all[train_idx].reset_index(drop=True)
        val_data_info = df_all[val_idx].reset_index(drop=True)
        test_data_info = df_all[test_idx].reset_index(drop=True)

        if self.mode == 'train':
            data_info = train_data_info
        elif self.mode == 'val':
            data_info = val_data_info
        elif self.mode == 'test':
            data_info = test_data_info
        else:
            data_info = df_all
        
        logger.info(f"Mode: {mode} Fold: {fold}")
        return data_info

    def pre_process(self, wav):
        
        if self.mode == 'test' or self.mode == 'val':
            return wav
        else:
            wav = self.pad_trunc(wav)
            return wav

    def __len__(self):
        return len(self.data_info)

    def __getitem__(self, idx):
        # Load the raw waveform from file using data_info to get filenames
        wav_path = os.path.join(self.get_audio_dir_path, self.data_info['filename'][idx]) + '.wav'
        wav, sample_rate = torchaudio.load(wav_path)

        # Preprocess the waveform (e.g., pad/truncate if needed)
        wav = self.pre_process(wav)

        # Apply Wav2Vec2 feature extractor
        inputs = feature_extractor(
            wav.squeeze().numpy(),  # Convert PyTorch tensor to numpy array
            sampling_rate=sample_rate,
            return_tensors="pt",  # Return PyTorch tensors
            padding=True  # Optionally pad to a fixed length
        )

        label = self.label[idx]

        # Return the processed input values and the label
        return {
            'input_values': inputs['input_values'].squeeze(0),  # Remove extra batch dimension
            'labels': torch.tensor(label, dtype=torch.long)}

In [13]:
# Instantiate datasets
train_dataset = Mydataset(mode='train', max_len=6, fold=FOLD)
val_dataset = Mydataset(mode='val', max_len=6, fold=FOLD)
test_dataset = Mydataset(mode='test', max_len=6, fold=FOLD)

logger.info("Dataset Loaded")

[2025-01-20 01:15:46,039][689674042.py][line:43][INFO] Mode: train Fold: 0
[2025-01-20 01:15:46,053][689674042.py][line:43][INFO] Mode: val Fold: 0
[2025-01-20 01:15:46,064][689674042.py][line:43][INFO] Mode: test Fold: 0
[2025-01-20 01:15:46,066][1194672651.py][line:6][INFO] Dataset Loaded


In [14]:
# Put and val test information into a dataframe for later use
data_info = test_dataset.data_info
test_dataframe = data_info[['filename', 'label']].copy()
test_dataframe['filepath'] = test_dataframe['filename'].apply(
    lambda x: os.path.join(test_dataset.get_audio_dir_path, f"{x}.wav"))

val_info = val_dataset.data_info
val_dataframe = val_info[['filename', 'label']].copy()
val_dataframe['filepath'] = val_dataframe['filename'].apply(
    lambda x: os.path.join(val_dataset.get_audio_dir_path, f"{x}.wav"))

## Perform Inference and Obtain Transcriptions

#### Function to clean up transcription to match what is required for compute_wer.py
- Remove full stop
- Remove all punctuations except apostrophe
- Change all to UPPERCASE

In [15]:
def format_transcription(ID, text_dict):
    """
    Format the transcription by:
      1. Removing all non-ASCII characters.
      2. Removing periods.
      3. Removing all punctuation except apostrophes.
      4. Converting to uppercase.
    """
    
    # Extract the raw text and keep a copy for comparison
    text = text_dict['text']
    original_text = text
    
    # 1. Remove all non-ASCII characters
    clean_text = ''.join(c for c in text if ord(c) < 128)
    
    # 2. Remove full stops (periods)
    clean_text = clean_text.replace('.', '')
    
    # 3. Remove all punctuations except apostrophes
    #    (everything that is not alphanumeric or apostrophe => replaced by space)
    clean_text = ''.join(char if char.isalnum() or char == "'" else ' ' 
                         for char in clean_text)
    
    # 4. Convert to uppercase
    clean_text = clean_text.upper()

    # Alert if any formatting was done
    # (i.e., if the final clean_text differs from the *original*, unmodified text)
    if clean_text != original_text:
        logger.info(f"Transcription of file {ID} has been formatted")
        logger.info(f"ORIGINAL: {original_text}")
        logger.info(f"FORMATTED: {clean_text}")
    
    return clean_text

In [16]:
def predict(dataframe, split):

    logger.info(f"----- {split} split -----")

    results = []
    total = dataframe.shape[0]
    count = 1

    # Iterate over each audio file in the test folder
    for index, row in dataframe.iterrows():

        # Display progress
        print(f'File {count} of {total}', end='\r')
        count += 1

        # Load audio file
        filename = row['filename'] + '.wav'
        audio_file = os.path.join(AUDIO_DIRECTORY, filename)
        audio, sample_rate = librosa.load(audio_file, sr = 16000)
        
        # Get transcriptions
        transcription = pipe({"array": audio, "sampling_rate": sample_rate})
        
        # Tokenize the input audio for speech-to-text model
        #input_features = S2T_processor(audio, return_tensors="pt", sampling_rate=16000, padding="longest").input_features
        #input_features = input_features.to('cuda')
        #predicted_ids = S2T_Model.generate(input_features)
        
        #transcription = S2T_processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
        
        # Important!
        # Batch size should be 1 for inference. I am not using a dataloader, and iterating per file, so my batch size is effectively one
        # Padding="longest" pads each file to the longest in each batch, but because my batch is 1, effectively no padding is done.
        # For clarify, padding="do_not_pad" is used
        
        # Obtain transcriptions
        #with torch.no_grad():
        #    S2T_logits = S2T_Model(input_values).logits
        #    predicted_ids = torch.argmax(S2T_logits, dim=-1)
        #    transcription = S2T_processor.batch_decode(predicted_ids)[0]
            
        # Perform cleaning
        formatted_transcription = format_transcription(count, transcription)

        # Extract the filename without the extension
        filename = os.path.splitext(os.path.basename(audio_file))[0]

        # Append the result to the list
        results.append([filename,
                        formatted_transcription])

    logger.info(f"Done processing {total} files")

    # Write the results to a CSV file
    global CSV_FILEPATH
    CSV_FILEPATH = os.path.join(OUTPUT_FOLDER, f"{split}-transcriptions.csv")

    with open(CSV_FILEPATH, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'transcription'])
        writer.writerows(results)

    logger.info(f"Transcriptions saved to {CSV_FILEPATH}")
    return CSV_FILEPATH

In [17]:
predict(val_dataframe, 'val')
predict(test_dataframe, 'test')

[2025-01-20 01:15:46,101][2267696535.py][line:3][INFO] ----- val split -----


C:\Users\ChenYi\anaconda3\envs\aai3001\Lib\site-packages\transformers\models\whisper\generation_whisper.py:496: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
C:\Users\ChenYi\anaconda3\envs\aai3001\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:599: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Passing a tuple of `pa

[2025-01-20 01:15:49,512][835302387.py][line:31][INFO] Transcription of file 3 has been formatted
[2025-01-20 01:15:49,512][835302387.py][line:32][INFO] ORIGINAL:  You're not sorry you came?
[2025-01-20 01:15:49,527][835302387.py][line:33][INFO] FORMATTED:  YOU'RE NOT SORRY YOU CAME 


[2025-01-20 01:15:49,814][835302387.py][line:31][INFO] Transcription of file 4 has been formatted
[2025-01-20 01:15:49,814][835302387.py][line:32][INFO] ORIGINAL:  Well...
[2025-01-20 01:15:49,830][835302387.py][line:33][INFO] FORMATTED:  WELL


[2025-01-20 01:15:50,129][835302387.py][line:31][INFO] Transcription of file 5 has been formatted
[2025-01-20 01:15:50,129][835302387.py][line:32][INFO] ORIGINAL:  What?
[2025-01-20 01:15:50,129][835302387.py][line:33][INFO] FORMATTED:  WHAT 


...

[2025-01-20 01:29:32,024][835302387.py][line:31][INFO] Transcription of file 526 has been formatted
[2025-01-20 01:29:32,025][835302387.py][line:32][INFO] ORIGINAL:  Oh, thank you, darling. The same applies to you, too. Except that if you so much as look at another woman, I'll kill you.
[2025-01-20 01:29:32,026][835302387.py][line:33][INFO] FORMATTED:  OH  THANK YOU  DARLING THE SAME APPLIES TO YOU  TOO EXCEPT THAT IF YOU SO MUCH AS LOOK AT ANOTHER WOMAN  I'LL KILL YOU


[2025-01-20 01:29:32,909][835302387.py][line:31][INFO] Transcription of file 527 has been formatted
[2025-01-20 01:29:32,910][835302387.py][line:32][INFO] ORIGINAL:  Oh, Charles! Oh, that was his name. He did Wiggles so beautifully.
[2025-01-20 01:29:32,911][835302387.py][line:33][INFO] FORMATTED:  OH  CHARLES  OH  THAT WAS HIS NAME HE DID WIGGLES SO BEAUTIFULLY


[2025-01-20 01:29:34,090][835302387.py][line:31][INFO] Transcription of file 528 has been formatted
[2025-01-20 01:29:34,091][835302387.py][line:32][INFO] ORIGINAL:  The manager came in, he found us rolling about the floor, biting and scratching like panthers. How utterly, utterly ridiculous.
[2025-01-20 01:29:34,092][835302387.py][line:33][INFO] FORMATTED:  THE MANAGER CAME IN  HE FOUND US ROLLING ABOUT THE FLOOR  BITING AND SCRATCHING LIKE PANTHERS HOW UTTERLY  UTTERLY RIDICULOUS


[2025-01-20 01:29:34,763][835302387.py][line:31][INFO] Transcription of file 529 has been formatted
[2025-01-20 01:29:34,764][835302387.py][line:32][INFO] ORIGINAL:  We were very much younger then. And very much sillier.
[2025-01-20 01:29:34,765][835302387.py][line:33][INFO] FORMATTED:  WE WERE VERY MUCH YOUNGER THEN AND VERY MUCH SILLIER
[2025-01-20 01:29:34,767][2267696535.py][line:52][INFO] Done processing 528 files
[2025-01-20 01:29:34,769][2267696535.py][line:63][INFO] Transcriptions saved to ./fold_0\test-transcriptions.csv


'./fold_0\\test-transcriptions.csv'

## Convert CSV into TXT file
- Optional step to convert into TXT file for use in compute_wer.py
- This function can also be found in notebooks/TranscriptionFormatter.ipynb, where you can optionaly convert each fold at once

In [18]:
# Input and output file paths
input_csv = CSV_FILEPATH  # Replace with your CSV file path

In [19]:
# Open the CSV file and read its content
def convert_csv_to_txt(input_csv, output_txt):
    with open(input_csv, mode="r", encoding="utf-8") as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader, None)  # Skip header if present
        with open(output_txt, mode="w", encoding="utf-8") as txt_file:
            for row in csv_reader:
                # Combine ID and Transcript with a space separator
                txt_file.write(f"{row[0]} {row[1]}\n")


for split in ['val', 'test']:
    output_txt = OUTPUT_FOLDER + f"/{split}-transcriptions.txt"
    convert_csv_to_txt(input_csv, output_txt)
    print(f"Converted and formatted {input_csv} to {output_txt} for use in computing word error rate.")

Converted and formatted ./fold_0\test-transcriptions.csv to ./fold_0/val-transcriptions.txt for use in computing word error rate.
Converted and formatted ./fold_0\test-transcriptions.csv to ./fold_0/test-transcriptions.txt for use in computing word error rate.
